<a href="https://colab.research.google.com/github/tmtakashi/machine_learning_notebooks/blob/master/chokkan_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 第１章：ニューラルネットワークの基礎

## シンプルなネットワーク（入力層と出力層のみ）


In [0]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils 


# 再現性のためのシード
np.random.seed(1671)

# ハイパーパラメータ
NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2

# --------
#データの前処理
# --------

# データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 28x28の画像のreshape
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

# GPU計算のためにfloat32型に変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# ラベルをone-hotに変換
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# -------------
# モデル構築
# -------------
model = Sequential()
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED, )))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

# -------------------------
# Tensor Boardの設定
# -------------------------

import os 
from time import gmtime, strftime
from keras.callbacks import TensorBoard

def make_tensorboard(set_dir_name=''):
  tictoc = strftime("%a_ %d_ %b_ &Y %H_ %M_ %S", gmtime())
  directory_name = tictoc
  log_dir = set_dir_name + '_' + directory_name
  os.mkdir(log_dir)
  tensorboard = TensorBoard(log_dir=log_dir)
  return tensorboard

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V1')]

# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                7850      
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 2s 38us/step - loss: 1.3633 - acc: 0.6796 - val_loss: 0.8904 - val_acc: 0.8246
Epoch 2/200
48000/48000 [==============================] - 2s 33us/step - loss: 0.7913 - acc: 0.8272 - val_loss: 0.6572 - val_acc: 0.8546
Epoch 3/200
48000/48000 [==============================] - 2s 33us/step - loss: 0.6436 - acc: 0.8497 - val_loss: 0.5625 - val_acc: 0.8681
Epoch 4/200
48000/48000 [========

## 隠れ層の追加による精度向上
- 隠れ層を２つ追加
- エポック数を200 => 20にしても精度向上

In [0]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils 

# 再現性のためのシード
np.random.seed(1671)

# ハイパーパラメータ
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2

# --------
#データの前処理
# --------

# データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 28x28の画像のreshape
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

# GPU計算のためにfloat32型に変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# ラベルをone-hotに変換
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# -------------
# モデル構築
# -------------
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, )))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

# -------------------------
# Tensor Boardの設定
# -------------------------

import os 
from time import gmtime, strftime
from keras.callbacks import TensorBoard

def make_tensorboard(set_dir_name=''):
  tictoc = strftime("%a_ %d_ %b_ &Y %H_ %M_ %S", gmtime())
  directory_name = tictoc
  log_dir = set_dir_name + '_' + directory_name
  os.mkdir(log_dir)
  tensorboard = TensorBoard(log_dir=log_dir)
  return tensorboard

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V2')]

# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_12 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_______________________________

## Dropoutによる精度向上

In [0]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils 

# 再現性のためのシード
np.random.seed(1671)

# ハイパーパラメータ
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

# --------
#データの前処理
# --------

# データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 28x28の画像のreshape
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

# GPU計算のためにfloat32型に変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# ラベルをone-hotに変換
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# -------------
# モデル構築
# -------------
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, )))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

# -------------------------
# Tensor Boardの設定
# -------------------------

import os 
from time import gmtime, strftime
from keras.callbacks import TensorBoard

def make_tensorboard(set_dir_name=''):
  tictoc = strftime("%a_ %d_ %b_ &Y %H_ %M_ %S", gmtime())
  directory_name = tictoc
  log_dir = set_dir_name + '_' + directory_name
  os.mkdir(log_dir)
  tensorboard = TensorBoard(log_dir=log_dir)
  return tensorboard

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V3')]

# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_16 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_17 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1

学習回数を大幅に増やす

In [0]:
NB_EPOCH = 250

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, )))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])
# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_19 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_20 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                1290      
__________

## 最適化アルゴリズムの変更

In [0]:
# RMSpropに変更
from keras.optimizers import RMSprop, Adam

NB_EPOCH = 20
OPTIMIZER = RMSprop()

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, )))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V4')]
# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_26 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                1290      
__________

In [0]:
# Adamに変更
from keras.optimizers import RMSprop, Adam

NB_EPOCH = 20
OPTIMIZER = Adam()

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, )))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V4')]
# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_28 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_29 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1290      
__________

- Adamのほうが若干良い
- 少ないエポック数で高い精度が出せる

- ニューロン数、エポック数の増加はコスパ悪い

## 正則化
- モデルが複雑になると過学習を起こしやすくなる => 複雑度に対してペナルティを与える

In [0]:
from keras import regularizers
 
NB_EPOCH = 20
OPTIMIZER = Adam()

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED, ),
               kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN,
               kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES,
               kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, 
             metrics=['accuracy'])

callbacks = [make_tensorboard(set_dir_name='drive/My Drive/Colab Notebooks/chokkan_deeplearning/keras_MNIST_V4')]
# -----
# 学習
# -----
model.fit(X_train, Y_train, 
         batch_size=BATCH_SIZE, epochs=NB_EPOCH,
         callbacks=callbacks,
         verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# ------
# 評価
# ------

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_32 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_33 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 10)                1290      
__________

# 第３章：畳み込みニューラルネットワーク


## LeNetの実装
- 深い層ほどフィルター数を増やす

In [0]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

def lenet(input_shape, num_classes):
  
  model = Sequential()
  
  #  畳み込み層とプーリング層で特徴マップを抽出
  model.add(Conv2D(20, kernel_size=5, padding="same",
                  input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(50, kernel_size=5, padding="same",
                   activation='relu'))
  
  # 全結合層でクラス判定
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  return model

class MNISTDataset():
  
  def __init__(self):
    self.image_shape = (28, 28, 1)
    self.num_classes = 10
    
  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
    y_train, y_test = [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
    
    return x_train, y_train, x_test, y_test
  
  def preprocess(self, data, label_data=False):
    if label_data:
      # クラスをOne-hotに変換
      data = keras.utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype("float32")
      data /= 255
      shape = (data.shape[0], ) + self.image_shape # データセットの長さを頭に加える
      data = data.reshape(shape)
      
    return data
  
class Trainer():
  
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer,
                        metrics=["accuracy"])
    self.verbose =1
    self.log_dir = os.path.join("./", "logdir")
    
  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    self._target.fit(
    x_train, y_train,
    batch_size=batch_size, epochs=epochs,
    validation_split=validation_split,
    callbacks=[TensorBoard(log_dir=self.log_dir)],
    verbose=self.verbose
    )
    
dataset = MNISTDataset()

# モデル作成
model = lenet(dataset.image_shape, dataset.num_classes)

# 学習
x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy",
                 optimizer=Adam())
trainer.train(x_train, y_train, batch_size=128, epochs=12,
             validation_split=0.2)

# 評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 10s 216us/step - loss: 0.1594 - acc: 0.9512 - val_loss: 0.0588 - val_acc: 0.9830
Epoch 2/12
48000/48000 [==============================] - 8s 164us/step - loss: 0.0435 - acc: 0.9864 - val_loss: 0.0429 - val_acc: 0.9877
Epoch 3/12
48000/48000 [==============================] - 8s 163us/step - loss: 0.0278 - acc: 0.9911 - val_loss: 0.0401 - val_acc: 0.9878
Epoch 4/12
48000/48000 [==============================] - 8s 163us/step - loss: 0.0195 - acc: 0.9935 - val_loss: 0.0345 - val_acc: 0.9895
Epoch 5/12
48000/48000 [==============================] - 8s 160us/step - loss: 0.0136 - acc: 0.9957 - val_loss: 0.0347 - val_acc: 0.9905
Epoch 6/12
48000/48000 [==============================] - 8s 158us/step - loss: 0.0090 - acc: 0.9969 - val_loss: 0.0394 - val_acc: 0.9905
Epoch 7/12
48000/48000 [==============================] - 8s 163us/step - loss: 0.0077 - acc: 0.9975 - val_loss: 0.0428 -

## CIFAR-10の画像認識

In [0]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint

def network(input_shape, num_classes):
  
  model = Sequential()
  
  #  畳み込み層とプーリング層で特徴マップを抽出
  model.add(Conv2D(32, kernel_size=3, padding="same",
                  input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, padding="same",
                   activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  # 全結合層でクラス判定
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  return model

class CIFAR10Dataset():
  
  def __init__(self):
    self.image_shape = (32, 32, 3)
    self.num_classes = 10
    
  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
    y_train, y_test = [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
    
    return x_train, y_train, x_test, y_test
  
  def preprocess(self, data, label_data=False):
    if label_data:
      # クラスをOne-hotに変換
      data = keras.utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype("float32")
      data /= 255
      shape = (data.shape[0], ) + self.image_shape # データセットの長さを頭に加える
      data = data.reshape(shape)
      
    return data
  
class Trainer():
  
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer,
                        metrics=["accuracy"])
    self.verbose =1
    self.log_dir = os.path.join("./", "logdir")
    self.model_file_name = "model_file.hdf5"
    
  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    self._target.fit(
    x_train, y_train,
    batch_size=batch_size, epochs=epochs,
    validation_split=validation_split,
    callbacks=[TensorBoard(log_dir=self.log_dir),
              ModelCheckpoint(os.path.join(self.log_dir,
                                          self.model_file_name), save_best_only=True)],
    verbose=self.verbose
    )
    
dataset = CIFAR10Dataset()

# モデル作成
model = network(dataset.image_shape, dataset.num_classes)

# 学習
x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy",
                 optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12,
             validation_split=0.2)

# 評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

170500096/170498071 [==============================] - 43s 0us/step
Train on 40000 samples, validate on 10000 samples
Epoch 1/12
40000/40000 [==============================] - 8s 189us/step - loss: 1.7327 - acc: 0.3809 - val_loss: 1.4056 - val_acc: 0.5070
Epoch 2/12
40000/40000 [==============================] - 7s 170us/step - loss: 1.3467 - acc: 0.5179 - val_loss: 1.2112 - val_acc: 0.5765
Epoch 3/12
40000/40000 [==============================] - 6s 155us/step - loss: 1.1820 - acc: 0.5819 - val_loss: 1.1211 - val_acc: 0.6223
Epoch 4/12
40000/40000 [==============================] - 6s 150us/step - loss: 1.0702 - acc: 0.6219 - val_loss: 1.0898 - val_acc: 0.6282
Epoch 5/12
40000/40000 [==============================] - 6s 154us/step - loss: 0.9854 - acc: 0.6553 - val_loss: 0.9472 - val_acc: 0.6739
Epoch 6/12
40000/40000 [==============================] - 6s 162us/step - loss: 0.9210 - acc: 0.6801 - val_loss: 0.9214 - val_acc: 0.6837
Epoch 7/12
40000/40000 [==============================

### 畳み込み層の追加による改善

In [0]:
def network(input_shape, num_classes):
  
  model = Sequential()
  
  #  畳み込み層とプーリング層で特徴マップを抽出
  model.add(Conv2D(32, kernel_size=3, padding="same",
                  input_shape=input_shape, activation='relu'))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, padding="same",
                   activation='relu'))
  model.add(Conv2D(64, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  # 全結合層でクラス判定
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  return model

# モデル作成
model = network(dataset.image_shape, dataset.num_classes)

# 学習
x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy",
                 optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12,
             validation_split=0.2)

# 評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/12
40000/40000 [==============================] - 10s 247us/step - loss: 1.7982 - acc: 0.3513 - val_loss: 1.4907 - val_acc: 0.4629
Epoch 2/12
40000/40000 [==============================] - 9s 230us/step - loss: 1.3813 - acc: 0.5052 - val_loss: 1.2096 - val_acc: 0.5693
Epoch 3/12
40000/40000 [==============================] - 9s 233us/step - loss: 1.1574 - acc: 0.5963 - val_loss: 1.0858 - val_acc: 0.6195
Epoch 4/12
40000/40000 [==============================] - 10s 239us/step - loss: 1.0142 - acc: 0.6441 - val_loss: 0.9612 - val_acc: 0.6570
Epoch 5/12
40000/40000 [==============================] - 9s 232us/step - loss: 0.9000 - acc: 0.6845 - val_loss: 0.9354 - val_acc: 0.6768
Epoch 6/12
40000/40000 [==============================] - 9s 234us/step - loss: 0.8106 - acc: 0.7183 - val_loss: 0.8558 - val_acc: 0.7004
Epoch 7/12
40000/40000 [==============================] - 9s 234us/step - loss: 0.7437 - acc: 0.7420 - val_loss: 0.8056 

### Data Augumentationによる改善

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

class Trainer():
  
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer,
                        metrics=["accuracy"])
    self.verbose =1
    self.log_dir = os.path.join("drive/My Drive/Colab Notebooks/chokkan_deeplearning", "logdir")
    self.model_file_name = "model_file.hdf5"
    
  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
    
    datagen.fit(x_train)
    
    # 検証データのためのsplit
    indices = np.arange(x_train.shape[0])
    np.random.shuffle(indices)
    validation_size = int(x_train.shape[0] * validation_split)
    x_train, x_valid = x_train[indices[:-validation_size], :], x_train[indices[-validation_size:],:]
    y_train, y_valid = y_train[indices[:-validation_size], :], y_train[indices[-validation_size:],:]
    
    
    self._target.fit_generator(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_valid, y_valid),
    callbacks=[TensorBoard(log_dir=self.log_dir),
              ModelCheckpoint(os.path.join(self.log_dir,
                                          self.model_file_name), save_best_only=True)],
    verbose=self.verbose,
    workers=4
    )
    
# モデル作成
model = network(dataset.image_shape, dataset.num_classes)

# 学習
x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy",
                 optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=15,
             validation_split=0.2)

# 評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Epoch 1/15
312/312 [==============================] - 32s 103ms/step - loss: 1.8883 - acc: 0.3102 - val_loss: 1.4418 - val_acc: 0.4777
Epoch 2/15
312/312 [==============================] - 31s 98ms/step - loss: 1.5188 - acc: 0.4539 - val_loss: 1.2843 - val_acc: 0.5410
Epoch 3/15
312/312 [==============================] - 30s 98ms/step - loss: 1.3413 - acc: 0.5211 - val_loss: 1.1984 - val_acc: 0.5798
Epoch 4/15
312/312 [==============================] - 31s 98ms/step - loss: 1.2259 - acc: 0.5636 - val_loss: 1.0359 - val_acc: 0.6339
Epoch 5/15
312/312 [==============================] - 30s 97ms/step - loss: 1.1339 - acc: 0.5970 - val_loss: 1.0013 - val_acc: 0.6459
Epoch 6/15
312/312 [==============================] - 31s 98ms/step - loss: 1.0575 - acc: 0.6304 - val_loss: 0.9328 - val_acc: 0.6716
Epoch 7/15
312/312 [==============================] - 30s 98ms/step - loss: 1.0077 - acc: 0.6473 - val_loss: 0.8040 - val_acc: 0.7167
Epoch 8/15
312/312 [==============================] - 30s 97m

In [0]:
from pathlib import Path
import pathlib
import numpy as np
from PIL import Image
from keras.models import load_model

model_path = "logdir/model_file.hdf5"
images_folder = "drive/My Drive/Colab Notebooks/chokkan_deeplearning/sample_images"

#  モデルの読み込み
model = load_model(os.path.join("drive/My Drive/Colab Notebooks/chokkan_deeplearning", model_path))
image_shape = (32, 32, 3)

# 画像の読み込み
def crop_resize(image_path):
  image = Image.open(image_path)
  length = min(image.size)
  crop = image.crop((0, 0, length, length))
  resized = crop.resize(image_shape[:2])
  img = np.array(resized).astype("float32")
  img /= 255
  return img

folder = Path(images_folder)
image_paths = [str(f) for f in folder.glob("*.png")]
images = [crop_resize(p) for p in image_paths]
images = np.asarray(images)

predicted = model.predict_classes(images)

# 判定できなかった場合に出力
assert predicted[0] == 3, "image should be cat."
assert predicted[1] == 5, "image should be dog."

print("You can detect cat&dog!")

You can detect cat&dog!


## 大規模な画像認識のための非常に深いネットワーク

### VGG16

In [0]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
import keras.preprocessing.image as Image
import numpy as np

model = VGG16(weights="imagenet", include_top=True)

image_path = "drive/My Drive/Colab Notebooks/chokkan_deeplearning/sample_images_pretrain/steaming_train.png"
image = Image.load_img(image_path, target_size=(224, 224))
x = Image.img_to_array(image)
x = np.expand_dims(x, axis=0) # バッチサイズのための次元を追加
x = preprocess_input(x)

result = model.predict(x)
result = decode_predictions(result, top=3)[0]
print(result[0][1])

40960/35363 [==================================] - 0s 5us/step
steam_locomotive


### 学習済みモデルを特徴抽出器として活用する

In [0]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights="imagenet")
model = Model(inputs=base_model.input,
             outputs=base_model.get_layer("block4_pool").output)

img_path =  "drive/My Drive/Colab Notebooks/chokkan_deeplearning/sample_images_pretrain/elephant.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = Image.img_to_array(img)
x = np.expand_dims(x, axis=0) # バッチサイズのための次元を追加
x = preprocess_input(x)

result = model.predict(x)

### Inception-v3を用いた転移学習

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# 基本となる学習済みモデル
base_model = InceptionV3(weights="imagenet", include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 付け加えた層のみ学習
for layer in base_model.layers:
  layer.trainable = False
  
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

#model.fit_generator(...)

# 第４章：GANとWaveNet

外部ライブラリに依存し過ぎなので、https://github.com/eriklindernoren/Keras-GAN を参考にした

## DCGANによるMNIST生成

In [0]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class DCGAN():
  def __init__(self):
    # 入力サイズ
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 100
    
    optimizer = Adam(0.0002, 0.5)
    
    # 識別モデルの構築
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
                              optimizer=optimizer,
                              metrics=['accuracy'])
    
    # 生成モデルの構築
    self.generator = self.build_generator()
    
    # 生成モデルの入力は一様分布Z
    z = Input(shape=(self.latent_dim,))
    img = self.generator(z)
    
    #　結合モデルでは生成モデルのみ学習させる
    self.discriminator.trainable = False
    
    # 識別モデルの判定結果
    valid = self.discriminator(img)
    
    # 結合モデル
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def build_generator(self):
    
    model = Sequential()
    
    model.add(Dense(128*7*7, activation='relu', input_dim=self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    
    model.summary()
    
    noise = Input(shape=(self.latent_dim,))
    img = model(noise)
    
    return Model(noise, img)
  
  def build_discriminator(self):
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    img = Input(shape=self.img_shape)
    validity = model(img)
    
    return Model(img, validity)
  
  def train(self, iterations, batch_size=128, save_interval=50):
    
    # データセットの読み込み
    (X_train, _), (_, _) = mnist.load_data()
    
    # -1~1で正規化
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)
    
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for iteration in range(iterations):
      
      # ----------------------
      # 識別モデルの学習
      # ----------------------
      
      # 画像の取り出し
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      
      # ノイズ発生および画像生成
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      gen_imgs = self.generator.predict(noise)
      
      # 学習
      d_loss_real = self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      # ---------------------
      # 生成モデルの学習
      # ---------------------
      
      # 騙せたときに1を返してほしい
      g_loss = self.combined.train_on_batch(noise, valid)
     
      if iteration % save_interval == 0:
        print("%d[D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100*d_loss[1], g_loss))
        self.save_imgs(iteration)
      
  def save_imgs(self, iteration):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generator.predict(noise)
    
    # 画像を0-1にスケーリング
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
        axs[i, j].axis('off')
        cnt += 1
    fig.savefig("drive/My Drive/Colab Notebooks/chokkan_deeplearning/dcgan_mnist/mnist_%d.png" % iteration)
    plt.close()   

In [0]:
dcgan = DCGAN()
dcgan.train(iterations=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  sample_weight: User-provided `sample_weight` argument.


0[D loss: 1.147271, acc.: 39.06%] [G loss: 0.492921]
50[D loss: 0.789120, acc.: 48.44%] [G loss: 1.824549]
100[D loss: 0.819251, acc.: 56.25%] [G loss: 1.229795]
150[D loss: 0.924419, acc.: 43.75%] [G loss: 1.149137]
200[D loss: 0.794385, acc.: 56.25%] [G loss: 1.144859]
250[D loss: 0.783535, acc.: 54.69%] [G loss: 1.209517]
300[D loss: 0.796604, acc.: 48.44%] [G loss: 0.961871]
350[D loss: 0.750702, acc.: 54.69%] [G loss: 1.111688]
400[D loss: 0.660403, acc.: 64.06%] [G loss: 0.957798]
450[D loss: 0.838665, acc.: 43.75%] [G loss: 0.993033]
500[D loss: 0.769370, acc.: 60.94%] [G loss: 1.059499]
550[D loss: 0.759797, acc.: 57.81%] [G loss: 1.051556]
600[D loss: 0.673682, acc.: 59.38%] [G loss: 1.163917]
650[D loss: 0.835735, acc.: 37.50%] [G loss: 1.029230]
700[D loss: 0.755871, acc.: 53.12%] [G loss: 1.018878]
750[D loss: 0.674510, acc.: 59.38%] [G loss: 1.111059]
800[D loss: 0.705506, acc.: 56.25%] [G loss: 1.085922]
850[D loss: 0.694029, acc.: 56.25%] [G loss: 0.864191]
900[D loss: 0

## DCGANによるCIFAR10生成

In [0]:
from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.regularizers import l1_l2

import matplotlib.pyplot as plt

import numpy as np

class DCGAN_CIFAR():
  def __init__(self):
    # 入力サイズ
    self.img_rows = 32
    self.img_cols = 32
    self.channels = 3
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 100
    
    optimizer = Adam(0.0002, 0.5)
    
    # 識別モデルの構築
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
                              optimizer=optimizer,
                              metrics=['accuracy'])
    
    # 生成モデルの構築
    self.generator = self.build_generator()
    
    # 生成モデルの入力は一様分布Z
    z = Input(shape=(self.latent_dim,))
    img = self.generator(z)
    
    #　結合モデルでは生成モデルのみ学習させる
    self.discriminator.trainable = False
    
    # 識別モデルの判定結果
    valid = self.discriminator(img)
    
    # 結合モデル
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def build_generator(self):
    
    model = Sequential()
    
    model.add(Dense(128*8*8, activation='relu', input_dim=self.latent_dim))
    model.add(Reshape((8, 8, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    
    model.summary()
    
    noise = Input(shape=(self.latent_dim,))
    img = model(noise)
    
    return Model(noise, img)
  
  def build_discriminator(self):
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    img = Input(shape=self.img_shape)
    validity = model(img)
    
    return Model(img, validity)
  def train(self, iterations, batch_size=128, save_interval=50):
    
    # データセットの読み込み
    (X_train, _), (_, _) = cifar10.load_data()
    
    # -1~1で正規化
    X_train = X_train / 127.5 - 1.
    
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for iteration in range(iterations):
      
      # ----------------------
      # 識別モデルの学習
      # ----------------------
      
      # 画像の取り出し
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      
      # ノイズ発生および画像生成
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      gen_imgs = self.generator.predict(noise)
      
      # 学習
      d_loss_real = self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      # ---------------------
      # 生成モデルの学習
      # ---------------------
      
      # 騙せたときに1を返してほしい
      g_loss = self.combined.train_on_batch(noise, valid)
     
      if iteration % save_interval == 0:
        print("%d[D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100*d_loss[1], g_loss))
        self.save_imgs(iteration)
      
  def save_imgs(self, iteration):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generator.predict(noise)
    
    # 画像を0-1にスケーリング
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i, j].imshow(gen_imgs[cnt, :, :, :])
        axs[i, j].axis('off')
        cnt += 1
    fig.savefig("drive/My Drive/Colab Notebooks/chokkan_deeplearning/dcgan_cifar10/cifar_%d.png" % iteration)
    plt.close()   

In [0]:
dcgan = DCGAN_CIFAR()
dcgan.train(iterations=8000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_75 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 9, 9, 64)          0         
_________________________________________________________________
batch_normalization_44 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
leaky_re_lu_48 (LeakyReLU)   (None, 9, 9, 64)          0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0[D loss: 0.993627, acc.: 45.31%] [G loss: 0.490350]
50[D loss: 0.552495, acc.: 71.88%] [G loss: 1.068514]
100[D loss: 0.663663, acc.: 60.94%] [G loss: 1.919299]
150[D loss: 0.427101, acc.: 81.25%] [G loss: 2.052676]
200[D loss: 0.742191, acc.: 60.94%] [G loss: 1.548181]
250[D loss: 1.124821, acc.: 46.88%] [G loss: 1.213459]
300[D loss: 0.360107, acc.: 76.56%] [G loss: 2.002065]
350[D loss: 0.604558, acc.: 68.75%] [G loss: 1.639582]
400[D loss: 0.860352, acc.: 50.00%] [G loss: 1.087935]
450[D loss: 0.519880, acc.: 75.00%] [G loss: 1.436579]
500[D loss: 0.577512, acc.: 68.75%] [G loss: 1.500663]
550[D loss: 0.591313, acc.: 64.06%] [G loss: 1.499802]
600[D loss: 0.734878, acc.: 59.38%] [G loss: 1.341720]
650[D loss: 0.830927, acc.: 53.12%] [G loss: 0.786268]
700[D loss: 0.738660, acc.: 56.25%] [G loss: 1.365353]
750[D loss: 0.553006, acc.: 68.75%] [G loss: 0.978468]
800[D loss: 0.949355, acc.: 45.31%] [G loss: 1.007817]
850[D loss: 0.571557, acc.: 73.44%] [G loss: 0.875395]
900[D loss: 0

# 単語分散表現

## word2vec


### Skip-gram

In [0]:
from keras.layers import Dot, Dense, Reshape, Embedding
from keras.models import Sequential

vocab_size = 5000
embed_size = 300

# http://cookie-box.hatenablog.com/entry/2018/10/14/184801
class SkipGramDiscriminator():
  def __init__(self, vocab_size, embed_size):
    self.vocab_size = vocab_size #  語彙数
    self.embed_size = embed_size # 埋め込み次元数
  def create_model(self):
    # 中心語ID => 中心語数値ベクトル表現
    x0 = Input(shape=(1, ))
    y0 = Embedding(self.vocab_size, self.embed_size,
                  embeddings_initializer='glorot_uniform')(x0)
    y0 = Reshape((self.embed_size, ))(y0)
    self.word_embedder = Model(x0, y0)
    # 文脈語ID => 文脈語数値ベクトル表現
    x1 = Input(shape=(1, ))
    y1 = Embedding(self.vocab_size, self.embed_size,
                  embeddings_initializer='glorot_uniform')(x1)
    y1 = Reshape((self.embed_size, ))(y1)
    self.context_embedder = Model(x1, y1)
    # 内積 => ロジスティック回帰
    y = Dot(axes=-1)([y0, y1])
    y = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(y)
    self.discriminator = Model(inputs=[x0, x1], outputs=y)
    self.discriminator.compile(loss='mean_squared_error', optimizer='adam')
    self.discriminator.summary()
    
from keras.preprocessing.text import * 
from keras.preprocessing.sequence import skipgrams

text = "I love green eggs and ham ."

# 各単語を整数IDにマッピングする辞書を作成
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

wids = [word2id[w] for w in text_to_word_sequence(text)]
pairs, labels = skipgrams(wids, len(word2id), window_size=1)

print(len(pairs), len(labels))
for i in range(10):
  print("({:s} ({:d}), {:s}({:d})) -> {:d}".format(
            id2word[pairs[i][0]], pairs[i][0],
            id2word[pairs[i][1]], pairs[i][1],
            labels[i]))
  
sg = SkipGramDiscriminator(6, 3)
sg.create_model()
x0_samples = np.array([[1], [4], [1], [4], [2]]) # 中心語： love,and,love,and,green
x1_samples = np.array([[0], [5], [2], [2], [2]]) # 文脈語： i,ham,green,green,green
y_samples = sg.discriminator.predict([x0_samples, x1_samples])
print(y_samples) # 中心語と文脈語のペアであるかどうかの判定結果（学習まだ）

print('中心語の数値ベクトル表現は中心語の Embedding 層の重みそのもの\n', sg.word_embedder.get_weights())

# IDから数値ベクトル表現を取り出せることの確認
print('i の数値ベクトル表現: ', sg.word_embedder.predict([[0]]))
print('love の数値ベクトル表現', sg.word_embedder.predict([[1]])) 

20 20
(ham (6), eggs(4)) -> 0
(and (5), eggs(4)) -> 1
(i (1), i(1)) -> 0
(ham (6), and(5)) -> 1
(eggs (4), green(3)) -> 1
(love (2), eggs(4)) -> 0
(eggs (4), love(2)) -> 0
(eggs (4), and(5)) -> 1
(love (2), green(3)) -> 1
(love (2), i(1)) -> 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 1, 3)         18          input_32[0][0]                   
_______________________________________________________________

### CBOW

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Lambda, Embedding
import keras.backend as K

vocab_size = 5000
embed_size = 300
window_size = 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embed_size,
                                        embeddings_initializer='glorot_uniform',
                                        input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size, )))
model.add(Dense(vocab_size, kernel_initializer='glorot_uniform', activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

### gensimを使って実装

In [0]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 1.8MB/s 
    100% |████████████████████████████████| 1.4MB 15.9MB/s 
    100% |████████████████████████████████| 133kB 27.5MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 5.1MB 7.1MB/s 
    100% |████████████████████████████████| 552kB 26.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
import logging
import os

from gensim.models import word2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                    level=logging.INFO)

DATA_DIR = os.path.join('drive/My Drive/Colab Notebooks/chokkan_deeplearning', "data")
sentences = word2vec.Text8Corpus(os.path.join(DATA_DIR, "text8"), 50)
model = word2vec.Word2Vec(sentences, size=300, min_count=30)

model.init_sims(replace=True)
model.save("word2vec_gensim.bin")

2018-12-09 08:04:20,737 : INFO : collecting all words and their counts
2018-12-09 08:04:21,466 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-09 08:04:21,709 : INFO : PROGRESS: at sentence #10000, processed 500000 words, keeping 33463 word types
2018-12-09 08:04:21,952 : INFO : PROGRESS: at sentence #20000, processed 1000000 words, keeping 52754 word types
2018-12-09 08:04:22,174 : INFO : PROGRESS: at sentence #30000, processed 1500000 words, keeping 65588 word types
2018-12-09 08:04:22,400 : INFO : PROGRESS: at sentence #40000, processed 2000000 words, keeping 78382 word types
2018-12-09 08:04:22,619 : INFO : PROGRESS: at sentence #50000, processed 2500000 words, keeping 88007 word types
2018-12-09 08:04:22,828 : INFO : PROGRESS: at sentence #60000, processed 3000000 words, keeping 96644 word types
2018-12-09 08:04:23,069 : INFO : PROGRESS: at sentence #70000, processed 3500000 words, keeping 104308 word types
2018-12-09 08:04:23,370 : INFO : PROGRE

In [0]:
model.init_sims(replace=True)
model.save("drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin")
model = word2vec.Word2Vec.load('drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin')

2018-12-09 08:11:33,430 : INFO : precomputing L2-norms of word weight vectors
2018-12-09 08:11:33,670 : INFO : saving Word2Vec object under drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin, separately None
2018-12-09 08:11:33,672 : INFO : not storing attribute vectors_norm
2018-12-09 08:11:33,674 : INFO : not storing attribute cum_table
2018-12-09 08:11:34,433 : INFO : saved drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin
2018-12-09 08:11:34,435 : INFO : loading Word2Vec object from drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin
2018-12-09 08:11:35,001 : INFO : loading wv recursively from drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsim.bin.wv.* with mmap=None
2018-12-09 08:11:35,002 : INFO : setting ignored attribute vectors_norm to None
2018-12-09 08:11:35,003 : INFO : loading vocabulary recursively from drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/word2vec_gemsi

In [0]:
list(model.wv.vocab.keys())[0:10]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against']

In [0]:
model["woman"]

In [0]:
model.most_similar("woman")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-12-09 08:12:45,755 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('child', 0.7281472682952881),
 ('girl', 0.7064403295516968),
 ('man', 0.6807918548583984),
 ('lover', 0.6249527931213379),
 ('person', 0.617744505405426),
 ('lady', 0.6149004697799683),
 ('herself', 0.614129900932312),
 ('prostitute', 0.6023421287536621),
 ('daughter', 0.5937902331352234),
 ('baby', 0.5883060693740845)]

In [0]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.6095590591430664),
 ('throne', 0.5634428262710571),
 ('princess', 0.5530728101730347),
 ('elizabeth', 0.5459858179092407),
 ('emperor', 0.5451050400733948),
 ('prince', 0.5418133735656738),
 ('sigismund', 0.5339542627334595),
 ('daughter', 0.5274652242660522),
 ('isabella', 0.5246189832687378),
 ('empress', 0.5207471251487732)]

In [0]:
model.similarity("girl", "woman")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.70644027

## Glove

### ゼロから分散表現を学習する

In [0]:
import os
import collections

from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

np.random.seed(42)

Using TensorFlow backend.


In [0]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
INPUT_FILE = "drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/training.txt"
LOG_DIR = "drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/logs"
VOCAB_SIZE = 5000
EMBED_SIZE = 100
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 20

counter = collections.Counter()
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
  maxlen = 0
  for line in fin:
    _, sent = line.strip().split("\t")
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    maxlen = max(maxlen, len(words))
    for word in words:
      counter[word] += 1

word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
  word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

# 第６章：リカレントニューラルネットワーク

In [0]:
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
import numpy as np
import codecs

In [0]:
INPUT_FILE = "drive/My Drive/Colab Notebooks/chokkan_deeplearning/data/alice_in_wonderland.txt"

with codecs.open(INPUT_FILE, "r", encoding="utf-8") as f:
  lines = [line.strip().lower() for line in f
               if len(line) !=0]
  text = " ".join(lines)
  
chars = set(text)
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [0]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
  input_chars.append(text[i:i + SEQLEN])
  label_chars.append(text[i + SEQLEN])
  


In [0]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
  for j, ch in enumerate(input_char):
    X[i, j, char2index[ch]] = 1
  y[i, char2index[label_chars[i]]] = 1
  


In [0]:
HIDDEN_SIZE =128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                   input_shape=(SEQLEN, nb_chars),
                   unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='rmsprop')

In [0]:
for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  model.fit(X, y, batch_size=BATCH_SIZE,
           epochs=NUM_EPOCHS_PER_ITERATION)
  
  test_idx = np.random.randint(len(input_chars))
  test_chars = input_chars[test_idx]
  print("Generating from seed: %s" %(test_chars))
  print(test_chars, end="")
  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for i, ch in enumerate(test_chars):
      Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred, end="")
    test_chars = test_chars[1:] + ypred
  print()

Iteration #: 0
Epoch 1/1
162739/162739 [==============================] - 9s 54us/step - loss: 1.7773
Generating from seed: harp kick,
harp kick, and the gryphon a done of the she said alice so the grown in a long the mart of the reat the said a
Iteration #: 1
Epoch 1/1
162739/162739 [==============================] - 9s 54us/step - loss: 1.7252
Generating from seed: ?’  ‘yes,’
?’  ‘yes,’ said the could not the rouse said the could not the rouse said the could not the rouse said the cou
Iteration #: 2
Epoch 1/1
162739/162739 [==============================] - 9s 54us/step - loss: 1.6835
Generating from seed: 4 and the 
4 and the groped and the said the dormouse of the project gutenberg-tm enechion was she was she was she was sh
Iteration #: 3
Epoch 1/1
162739/162739 [==============================] - 9s 54us/step - loss: 1.6473
Generating from seed: ven if i f
ven if i for the rook to the mork to the the rook to the mork to the the rook to the mork to the the rook to t
Iteration #: 4
E

## LSTMで評判分析

In [0]:
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
import os
import codecs

In [0]:
!pip install nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
with codecs.open("umich-sentiment-train.txt",
                "r", 'utf-8') as ftrain:
  for line in ftrain:
    label, sentence = line.strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    maxlen = max(maxlen, len(words))
    for word in words:
      word_freqs[word] += 1
    num_recs += 1

In [0]:
print(maxlen)

42


In [0]:
print(len(word_freqs))

2328


In [0]:
MAX_FEATURES = 2000
MAX_SENTENCE_LENGTH = 40
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2